In [63]:
import numpy
import cv2
import copy

X_VAL = 1
Y_VAL = 0
F_VAL = 2
PAR_VAL = 3
G_VAL = 4
H_VAL = 5
MATRIX_HEIGHT = 10
MATRIX_WIDTH = 10
direction = None

In [2]:
'''
Priority Queue class taken from GeeksForGeeks
I only copied the delete, insert, isEmpty, __init__, and __str__ methods
https://www.geeksforgeeks.org/priority-queue-in-python/
'''
class PriorityQueue(object): 
    def __init__(self): 
        self.queue = [] 
  
    def __str__(self): 
        return ' '.join([str(i) for i in self.queue]) 
  
    # for checking if the queue is empty 
    def isEmpty(self): 
        return self.queue == []
  
    # for inserting an element in the queue 
    def insert(self, data): 
        self.queue.append(data) 
        
    def find_parent(self,data):
        for i in self.queue:
            if data == [i[Y_VAL], i[X_VAL], i[F_VAL]]:
                return i
            
    def lowest_similar(self, data):
        lowest = None
        #print("ls_data:", data)
        for i in self.queue:
            if data[X_VAL] == i[X_VAL] and data[Y_VAL] == i[Y_VAL]:
                if lowest == None or lowest > i[F_VAL]:
                    lowest = i[F_VAL]
                    
        #print("Queue:", self.queue)
        #print("Lowest:", lowest)
        return lowest
    
    def delete(self): 
        try: 
            min_i= 0
            for i in range(len(self.queue)): 
                if self.queue[i][F_VAL] < self.queue[min_i][F_VAL]: 
                    min_i = i 
            item = self.queue[min_i] 
            del self.queue[min_i] 
            return item 
        except IndexError: 
            print() 
            exit() 

In [3]:
matrix = cv2.imread("test3.png", 1)

In [4]:
def find_start(matrix):
    for i in range(0, len(matrix), 50):
        for j in range(0, len(matrix[i]), 50):
            if(matrix[i][j][0] > 0):
                start = format_coordinates(j, i)
    return start

In [5]:
def find_end(matrix):
    for i in range(0, len(matrix), 50):
        for j in range(0, len(matrix[i]), 50):
            if(matrix[i][j][2] > 0):
                end = format_coordinates(j, i)
    return end 

In [78]:
def search(matrix, start, end, algo="UCS", heuristic="MH"):
    open_list = PriorityQueue()
    closed_list = PriorityQueue()
    start.append(1) #F(start)
    start.append([start[Y_VAL], start[X_VAL], 0])
    if algo == "A*":
        start.append(0) #g(start)
    open_list.insert(start)
    steps = 0
    value = 0
    print("Algorithm:", algo)
   
    while not open_list.isEmpty():
        #print("Open List: ", open_list)
        q = open_list.delete()
        steps += 1
        value += q[F_VAL]
        if q[X_VAL] == end[X_VAL] and q[Y_VAL] == end[Y_VAL]:
            closed_list.insert(q)
            break
        successors = generate_successors(q[X_VAL], q[Y_VAL])
        q_data = get_color_data(matrix, q[X_VAL], q[Y_VAL])
        #print(q_data)
        f = 0
        for successor in successors:
            g = 0
            f = 0
            successor.append(f)
            #successor.append([q[Y_VAL], q[X_VAL], q[F_VAL], q[PAR_VAL]])
            successor.append([q[Y_VAL], q[X_VAL], q[F_VAL]])
            successor_data = get_color_data(matrix, successor[X_VAL], successor[Y_VAL])
            if algo == "UCS":
                f = get_intensity(q_data, successor_data)
                f += q[F_VAL]
                #print(f)
            elif algo == "GBFS":
                f = calculate_heuristic(successor, end, heuristic)
            elif algo == "A*":
                g = get_intensity(q_data, successor_data) + q[G_VAL]
                h = calculate_heuristic(successor, end, heuristic)
                f = g + h
                print("G + H", g, "+", h)
                
                #f += q[F_VAL]
            successor[F_VAL] = f
            
            if algo == "A*":
                successor.append(g)
            
            will_add = True
            ol_low = open_list.lowest_similar(successor) 
            if not ol_low == None and ol_low <= f:
                will_add = False
            if will_add:
                cl_low = closed_list.lowest_similar(successor) 
                if not cl_low == None and cl_low <= f:
                    will_add = False
            
            if successor_data[0] == 0 and successor_data[1] == 0 and successor_data[2] == 0 :
                print("WALL", end= " ")
                will_add = False
            if will_add:
                print("Adding Successor:", successor)
                open_list.insert(successor)
            else:
                print("Will not add", successor)
        closed_list.insert(q)
        
    print("CLOSED LIST")
    print(closed_list)
    img = copy.copy(matrix)
    curr = q
    attempt_list = copy.deepcopy(closed_list)
    while not attempt_list.isEmpty():
        curr = attempt_list.delete()
        if len(curr) >= 4:
            x1 = curr[PAR_VAL][X_VAL] 
            y1 = curr[PAR_VAL][Y_VAL]
        else:
            x1 = start[X_VAL] 
            y1 = start[Y_VAL]
        x2 = curr[X_VAL] 
        y2 = curr[Y_VAL]
        print("Drawing Line (", x1, ",",y1, ")->(", x2, ",", y2, ")" )
        draw_line(img, line_convert(x1), line_convert(y1), 
                  line_convert(x2), line_convert(y2), color=(139,139,139))
    curr = q
    while curr != start:
        if len(curr) >= 4:
            x1 = curr[PAR_VAL][X_VAL] 
            y1 = curr[PAR_VAL][Y_VAL]
        else:
            x1 = start[X_VAL] 
            y1 = start[Y_VAL]
        x2 = curr[X_VAL] 
        y2 = curr[Y_VAL]
        print("Drawing Line (", x1, ",",y1, ")->(", x2, ",", y2, ")" )
        draw_line(img, line_convert(x1), line_convert(y1), 
                  line_convert(x2), line_convert(y2))
        curr = closed_list.find_parent(curr[PAR_VAL])
        

        
    print("Steps: ", steps)
    print("Value: ", value)
    if len(q) == 5:
        print("Total distance:", q[G_VAL])
    else:
        print("Total distance:", q[F_VAL])
    cv2.startWindowThread()
    cv2.imshow("image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    img = matrix
            
        
        
    
    
    

In [7]:
def line_convert(n):
    return (n * 50) + 25

In [8]:
def check_and_add(queue, successor):
    for i in queue:
        if successor[X_VAL] == i[X_VAL] and successor[Y_VAL] == i[Y_VAL]:
            if successor[F_VAL] < i[F_VAL]:
                arr = []
                arr[X_VAL] = successor[X_VAL]
                arr[Y_VAL] = successor[Y_VAL]
                arr[F_VAL] = successor[F_VAL]
                queue.insert(arr)
    return queue


In [9]:
def format_coordinates(x, y):
     return [y //50, x//50]

In [10]:
def convert_coordinates(x, y):
    return [y * 50, x * 50]

In [11]:
'''
returns color matrix where channels are BGR
'''
def get_color_data(matrix, x, y):
    return matrix[y * 50][x * 50]

In [12]:
def generate_successors(x, y):
    successors = []
    if y < MATRIX_HEIGHT - 1:
        successors.append([y + 1, x])
    if y > 0:
        successors.append([y - 1, x])
    if x < MATRIX_WIDTH -1:
        successors.append([y, x + 1])
    if x > 0:
        successors.append([y, x - 1])
    return successors 
        

In [13]:
def get_intensity(i, j):
    intensity = abs(i[1] - j[1])
    return intensity

In [14]:
def draw_line(image, x1, y1, x2, y2, color=(255,255,255)):
    img = cv2.line(image, (x1, y1), (x2, y2), color, 1)
    return img

In [76]:
def calculate_heuristic(curr, end, heuristic):
    global direction
    if heuristic == "MH":
        return calculate_manhattan(curr[X_VAL], curr[Y_VAL], end[X_VAL], end[Y_VAL])
    elif heuristic == "FSDD" or heuristic == "DSDD":
        if direction == None:
            if abs(curr[X_VAL] - end[X_VAL] ) > abs(curr[Y_VAL] - end[Y_VAL]):
                direction = "X"
            else:
                direction = "Y"
        if direction == "X":
            dim_dist = calculate_sdd(curr[X_VAL], end[X_VAL])
            if heuristic == "DSDD" and dim_dist == 0 and curr[PAR_VAL][X_VAL] == curr[X_VAL]:
                print("SWITCHING")
                return calculate_sdd(curr[Y_VAL], end[Y_VAL])
            else:
                print(dim_dist)
                return dim_dist
        else:
            dim_dist = calculate_sdd(curr[Y_VAL], end[Y_VAL] )
            if heuristic == "DSDD" and dim_dist == 0 and curr[PAR_VAL][Y_VAL] == curr[Y_VAL]:
                print("SWITCHING")
                return calculate_sdd(curr[X_VAL], end[X_VAL])
            else:
                print(dim_dist)
                return dim_dist
        
    elif heuristic == "DD":
        return max(abs(curr[X_VAL] - end[X_VAL]), abs(curr[Y_VAL] - end[Y_VAL]))
        
    

In [16]:
def calculate_sdd(c1, c2):
    return abs(c1 - c2)

In [17]:
def calculate_manhattan(x1, y1, x2, y2):
    return abs(x1 - x2) + abs(y1 - y2)

In [69]:
start = find_start(matrix)
end = find_end(matrix)
#Algo options = UCS, GBFS, A*
search(matrix, start, end, "UCS", heuristic="DD")

Algorithm: UCS
Adding Successor: [5, 4, 119, [4, 4, 1]]
Adding Successor: [3, 4, 13, [4, 4, 1]]
Adding Successor: [4, 5, 119, [4, 4, 1]]
Adding Successor: [4, 3, 13, [4, 4, 1]]
Will not add [4, 4, 257, [3, 4, 13]]
Adding Successor: [2, 4, 77, [3, 4, 13]]
Adding Successor: [3, 5, 31, [3, 4, 13]]
Adding Successor: [3, 3, 221, [3, 4, 13]]
Adding Successor: [5, 3, 31, [4, 3, 13]]
Will not add [3, 3, 221, [4, 3, 13]]
Will not add [4, 4, 257, [4, 3, 13]]
Adding Successor: [4, 2, 77, [4, 3, 13]]
Will not add [4, 5, 119, [3, 5, 31]]
Adding Successor: [2, 5, 191, [3, 5, 31]]
Adding Successor: [3, 6, 145, [3, 5, 31]]
Will not add [3, 4, 269, [3, 5, 31]]
Adding Successor: [6, 3, 145, [5, 3, 31]]
Will not add [4, 3, 269, [5, 3, 31]]
Will not add [5, 4, 119, [5, 3, 31]]
Adding Successor: [5, 2, 191, [5, 3, 31]]
Will not add [3, 4, 269, [2, 4, 77]]
Adding Successor: [1, 4, 315, [2, 4, 77]]
Will not add [2, 5, 191, [2, 4, 77]]
Adding Successor: [2, 3, 285, [2, 4, 77]]
Will not add [5, 2, 191, [4, 2, 

/home/gabloo/py_envs/PA1/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in ubyte_scalars
  


# Run Area

In [81]:
start = find_start(matrix)
end = find_end(matrix)
search(matrix, start, end, "GBFS", "DSDD")

Algorithm: GBFS
5
Adding Successor: [5, 4, 5, [4, 4, 1]]
3
Adding Successor: [3, 4, 3, [4, 4, 1]]
4
Adding Successor: [4, 5, 4, [4, 4, 1]]
4
Adding Successor: [4, 3, 4, [4, 4, 1]]
4
Will not add [4, 4, 4, [3, 4, 3]]
2
Adding Successor: [2, 4, 2, [3, 4, 3]]
3
Adding Successor: [3, 5, 3, [3, 4, 3]]
3
Adding Successor: [3, 3, 3, [3, 4, 3]]
3
Will not add [3, 4, 3, [2, 4, 2]]
1
Adding Successor: [1, 4, 1, [2, 4, 2]]
2
Adding Successor: [2, 5, 2, [2, 4, 2]]
2
Adding Successor: [2, 3, 2, [2, 4, 2]]
2
Will not add [2, 4, 2, [1, 4, 1]]
0
Adding Successor: [0, 4, 0, [1, 4, 1]]
1
Adding Successor: [1, 5, 1, [1, 4, 1]]
1
Adding Successor: [1, 3, 1, [1, 4, 1]]
1
Will not add [1, 4, 1, [0, 4, 0]]
SWITCHING
Adding Successor: [0, 5, 3, [0, 4, 0]]
SWITCHING
Adding Successor: [0, 3, 1, [0, 4, 0]]
2
Will not add [2, 5, 2, [1, 5, 1]]
0
Adding Successor: [0, 5, 0, [1, 5, 1]]
1
Adding Successor: [1, 6, 1, [1, 5, 1]]
1
Will not add [1, 4, 1, [1, 5, 1]]
1
Will not add [1, 5, 1, [0, 5, 0]]
SWITCHING
Adding Su

In [57]:
start = find_start(matrix)
end = find_end(matrix)
search(matrix, start, end, "GBFS", heuristic="MH")

Algorithm: GBFS
Adding Successor: [5, 4, 7, [4, 4, 1]]
Adding Successor: [3, 4, 5, [4, 4, 1]]
Adding Successor: [4, 5, 7, [4, 4, 1]]
Adding Successor: [4, 3, 5, [4, 4, 1]]
Will not add [4, 4, 6, [3, 4, 5]]
Adding Successor: [2, 4, 4, [3, 4, 5]]
Adding Successor: [3, 5, 6, [3, 4, 5]]
Adding Successor: [3, 3, 4, [3, 4, 5]]
Will not add [3, 4, 5, [2, 4, 4]]
Adding Successor: [1, 4, 3, [2, 4, 4]]
Adding Successor: [2, 5, 5, [2, 4, 4]]
Adding Successor: [2, 3, 3, [2, 4, 4]]
Will not add [2, 4, 4, [1, 4, 3]]
Adding Successor: [0, 4, 2, [1, 4, 3]]
Adding Successor: [1, 5, 4, [1, 4, 3]]
Adding Successor: [1, 3, 2, [1, 4, 3]]
Will not add [1, 4, 3, [0, 4, 2]]
Adding Successor: [0, 5, 3, [0, 4, 2]]
Adding Successor: [0, 3, 1, [0, 4, 2]]
Will not add [1, 3, 2, [0, 3, 1]]
Will not add [0, 4, 2, [0, 3, 1]]
Adding Successor: [0, 2, 0, [0, 3, 1]]
CLOSED LIST
[4, 4, 1, [4, 4, 0]] [3, 4, 5, [4, 4, 1]] [2, 4, 4, [3, 4, 5]] [1, 4, 3, [2, 4, 4]] [0, 4, 2, [1, 4, 3]] [0, 3, 1, [0, 4, 2]] [0, 2, 0, [0, 3, 1